# Beam center finder

In SANS experiments, it is essential to find the center of the scattering pattern in order to allow symmetric summation of the scattering intensity around the beam (i.e. computing a one-dimensional $I(Q)$).
As detector panels can move, the center of the beam will not always be located at the same place on the detector panel from one experiment to the next.

Here we describe the beam center finding algorithm,
which uses a combination of a center-of-mass calculation and an iterative refinement on a computed scattering cross-section to find the center of the scattering pattern.

In [ ]:
import scipp as sc
from scipp.constants import g
from ess import loki, sans
from ess.logging import configure_workflow
import scippneutron as scn
import plopp as pp
pp.patch_scipp()

In [ ]:
logger = configure_workflow('sans_beam_center_finder', filename='sans.log')

We begin by setting some parameters relevant to the current sample.

In [ ]:
# Include effects of gravity?
gravity = True

# Wavelength binning
wavelength_bins = sc.linspace(dim='wavelength', start=2.0, stop=16.0, num=141, unit='angstrom')

# Define Q binning
q_bins = sc.linspace('Q', 0.02, 0.25, 65, unit='1/angstrom')

# Define coordinate transformation graph
graph = sans.conversions.sans_elastic(gravity=gravity)

Next we load the data files for the sample and direct runs:

In [ ]:
sample = loki.io.load_sans2d(filename=loki.data.get_path('SANS2D00063114.hdf5'))
direct = loki.io.load_sans2d(filename=loki.data.get_path('SANS2D00063091.hdf5'))
# Add X, Y coordinates
sample.coords['x'] = sample.coords['position'].fields.x
sample.coords['y'] = sample.coords['position'].fields.y
# Add gravity coordinate
sample.coords["gravity"] = sc.vector(value=[0, -1, 0]) * g

### Masking bad pixels

We create a quick image of the data (summing along the `tof` dimension) to inspect its contents.
We see a diffuse scattering pattern, centered around a dark patch with an arm to the north-east; this is the sample holder.
It is clear that the sample and the beam are not in the center of the panel, which is marked by the red dot

In [ ]:
image = sample.sum('tof').copy().hist(y=120, x=128)
p = image.plot(norm='log', aspect='equal')
p.ax.plot(0, 0, 'o', color='red', ms=5)
p

To avoid skew in future comparisons of integrated intensities between the different regions of the detector panel,
we mask out the sample holder, using a combination of a low-counts threshold and pixel locations.

In [ ]:
summed = sample.sum('tof')
holder_mask = (summed.data < sc.scalar(100, unit='counts')) & \
              (sample.coords['x'] > sc.scalar(0, unit='m')) & \
              (sample.coords['x'] < sc.scalar(0.42, unit='m')) & \
              (sample.coords['y'] < sc.scalar(0.05, unit='m')) & \
              (sample.coords['y'] > sc.scalar(-0.15, unit='m'))
sample.masks['holder_mask'] = holder_mask

We look at the image again, to verify we have masked the desired region
(note that the part of the arm furthest away from the sample has not been masked, but this will not matter further down as more masks will be added).

In [ ]:
image = sample.sum('tof').copy().hist(y=120, x=128)
p = image.plot(norm='log', aspect='equal')
p.ax.plot(0, 0, 'o', color='red', ms=5)
p

## Description of the procedure

The procedure to determine the precise location of the beam center is the following:

1. obtain an initial guess by computing the center-of-mass of the pixels, weighted by the counts on each pixel
2. from that initial guess, divide the panel into 4 quadrants
3. compute $I(Q)$ inside each quadrant and compute the residual difference between all 4 quadrants
4. iteratively move the centre position and repeat 2. and 3. until all 4 $I(Q)$ curves lie on top of each other

## Initial guess: center-of-mass calculation

Computing the center-of-mass is straightforward using the vector `position` coordinate.

In [ ]:
# First sum the data along the 'tof' dimension
summed = sample.sum('tof')

# The weights are just the data counts in each pixel
weights = sc.values(summed.data)

# The center-of-mass is simply the weighted mean of the positions
com = sc.sum(summed.coords['position'] * weights) / weights.sum()

# We compute the shift between the incident beam direction and the center-of-mass
incident_beam = sample.transform_coords('incident_beam', graph=graph).coords['incident_beam']
n_beam = incident_beam / sc.norm(incident_beam)
com_shift = com - sc.dot(com, n_beam) * n_beam
com_shift

We can now plot the center-of-mass on the same image as before:

In [ ]:
xc = com_shift.fields.x
yc = com_shift.fields.y
p = image.plot(norm='log', aspect='equal')
p.ax.plot(xc.value, yc.value, 'o', color='red', ms=5)
p

## Making 4 quadrants

We divide the panel into 4 quadrants.

In [ ]:
p = image.plot(norm='log', aspect='equal')
p.ax.plot(xc.value, yc.value, 'o', color='red', ms=5)
p.ax.axvline(xc.value, color='cyan')
p.ax.axhline(yc.value, color='cyan')
dx = 0.25
p.ax.text(xc.value + dx, yc.value + dx, 'North-East', ha='center', va='center')
p.ax.text(xc.value - dx, yc.value + dx, 'North-West', ha='center', va='center')
p.ax.text(xc.value - dx, yc.value - dx, 'South-East', ha='center', va='center')
p.ax.text(xc.value + dx, yc.value - dx, 'South-West', ha='center', va='center')
p

## Adding a circular mask

It is evident from the figure above that some quadrants (the ones to the West) contain more pixels that others.
They also extend further away from the center, which means that more pixels can contribute to a given $Q$ bin.
To avoid introducing such bias when searching for the beam center, we add a circular mask onto the detector panel.

In [ ]:
masking_radius = sc.scalar(0.35, unit='m')

r = sc.sqrt(sc.midpoints(image.coords['x'] - xc)**2 +
            sc.midpoints(image.coords['y'] - yc)**2)
image.masks['circle'] = r > masking_radius

p = image.plot(norm='log', aspect='equal')
p.ax.plot(xc.value, yc.value, 'o', color='red', ms=5)
p.ax.axvline(xc.value, color='cyan')
p.ax.axhline(yc.value, color='cyan')
p

## Converting to $Q$ inside each quadrant

We now perform a full [$I(Q)$ reduction](../../instruments/loki/sans2d_to_I_of_Q.ipynb) inside each quadrant[$^1$](#footnote1).
The reduction involves computing a normalizing term which, for the most part, does not depend on pixel positions.
We therefore compute this once, before starting iterations to refine the position of the center.

### First compute denominator to avoid loop over expensive compute

In [ ]:
# Extract monitor data
sample_monitors = {
    'incident': sample.attrs["monitor2"].value,
    'transmission': sample.attrs["monitor4"].value
}
direct_monitors = {
    'incident': direct.attrs["monitor2"].value,
    'transmission': direct.attrs["monitor4"].value
}
# Define the range where monitor data is considered not to be noise
non_background_range = sc.array(dims=['wavelength'], values=[0.7, 17.1], unit='angstrom')
# Pre-process monitor data
sample_monitors = sans.i_of_q.preprocess_monitor_data(
    sample_monitors,
    non_background_range=non_background_range,
    wavelength_bins=wavelength_bins)
direct_monitors = sans.i_of_q.preprocess_monitor_data(
    direct_monitors,
    non_background_range=non_background_range,
    wavelength_bins=wavelength_bins)

# Denominator is defined as:
# pixel_solid_angles * Sample_T_monitor * Direct_I_monitor / Direct_T_monitor
solid_angle = sans.normalization.solid_angle_of_rectangular_pixels(
        sample,
        pixel_width=sample.coords['pixel_width'],
        pixel_height=sample.coords['pixel_height'])

denominator = (solid_angle * sample_monitors['transmission'] *
               direct_monitors['incident'] / direct_monitors['transmission'])
# Convert wavelength coordinate to midpoints for future histogramming
denominator.coords['wavelength'] = sc.midpoints(denominator.coords['wavelength'])
denominator

### Compute $I(Q)$ inside the 4 quadrants

We begin by defining several parameters which are required to compute $I(Q)$.

In [ ]:
# Define 4 phi bins
pi = sc.constants.pi.value
phi_bins = sc.linspace('phi', -pi, pi, 5, unit='rad')

# Name the quadrants
quadrants = ['south-west', 'south-east', 'north-east', 'north-west']

# Define a single wavelength band
wavelength_bands = sc.concat(
        [wavelength_bins.min(), wavelength_bins.max()], dim='wavelength')

We now define a function which will apply the center offset to the pixel coordinates,
and compute $I(Q)$ inside each quadrant.

In [ ]:
def to_q(xy, sample, denominator, graph, q_bins, masking_radius, gravity, wavelength_bands):
    data = sample.copy(deep=False)
    data.coords['position'] = sample.coords['position'].copy(deep=True)

    # Offset the position according to the initial guess from the center-of-mass
    coords = data.transform_coords(['cyl_x_unit_vector', 'cyl_y_unit_vector'], graph=graph).coords
    center = xy[0] * coords['cyl_x_unit_vector'] + xy[1] * coords['cyl_y_unit_vector']
    center.unit = data.coords['position'].unit
    data.coords['position'] -= center
    
    # Add the circular mask
    coords = data.transform_coords(['cylindrical_x', 'cylindrical_y'], graph=graph).coords
    r = sc.sqrt(coords['cylindrical_x']**2 +
                coords['cylindrical_y']**2)
    data.masks['circle'] = r > masking_radius
    
    # Insert a copy of coords needed for conversion to Q.
    for c in ['position', 'sample_position', 'source_position']:
        denominator.coords[c] = data.coords[c]
    denominator.masks['circle'] = data.masks['circle']
    
    phi = data.transform_coords('phi', graph=graph,
                                keep_intermediate=False,
                                keep_inputs=False).coords['phi']

    out = {}
    for i, quad in enumerate(quadrants):
        # Select pixels based on phi
        sel = (phi >= phi_bins[i]) & (phi < phi_bins[i+1])
        # Data counts into Q bins
        data_q = sans.i_of_q.convert_to_q_and_merge_spectra(
            data=data[sel],
            graph=graph,
            q_bins=q_bins,
            gravity=gravity,
            wavelength_bands=wavelength_bands)
        # Denominator counts into Q bins
        denominator_q = sans.i_of_q.convert_to_q_and_merge_spectra(
            data=denominator[sel],
            graph=graph,
            q_bins=q_bins,
            gravity=gravity,
            wavelength_bands=wavelength_bands)
        # Normalize        
        out[quad] = sans.normalization.normalize(numerator=data_q, denominator=denominator_q).hist()
    return out

Finally, we run the computation for all quadrants:

In [ ]:
grouped = to_q(xy=[com_shift.fields.x.value, com_shift.fields.y.value],
               sample=sample,
               denominator=denominator,
               graph=graph,
               q_bins=q_bins,
               masking_radius=masking_radius,
               gravity=gravity,
               wavelength_bands=wavelength_bands)

We can now plot on the same figure all 4 $I(Q)$ curves for each quadrant.

In [ ]:
pp.plot(grouped, norm='log')

As we can see, the overlap between the curves from the 4 quadrants is not satisfactory.
We will now use an iterative procedure to improve our initial guess, until a good overlap between the curves is found.

For this, we first define a cost function, which gives us an idea of how good the overlap is:

$$
\text{cost} = \sum_{Q}\frac{\sum_{i=1}^{i=4} \left|I(Q)_{i} - \overline{I}(Q)\right|}{\overline{I}(Q)} ~,
$$

where $\overline{I}(Q)$ is the mean intensity of the 4 quadrants as a function of $Q$.

In [ ]:
def cost(xy, sample, denominator, graph, q_bins, masking_radius, gravity, wavelength_bands):
    data_q = to_q(xy=xy,
                  sample=sample,
                  denominator=denominator,
                  graph=graph,
                  q_bins=q_bins,
                  masking_radius=masking_radius,
                  gravity=gravity,
                  wavelength_bands=wavelength_bands)
    all_q = sc.concat(list(data_q.values()), dim='quadrant')
    ref = sc.values(all_q.mean('quadrant'))
    c = sc.abs(all_q - ref) / ref
    out = c.sum().value
    print(xy, out)
    return out

Next, we use Scipy's [`minimize`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html)
utility from the `optimize` module, to iteratively minimize the computed cost.

In [ ]:
from scipy.optimize import minimize

# The minimizer works best if given bounds, which are the bounds of our detector panel
x = sample.coords['position'].fields.x
y = sample.coords['position'].fields.y
res = minimize(cost,
               x0=[com_shift.fields.x.value, com_shift.fields.y.value],
               args=(sample, denominator, graph, q_bins, masking_radius, gravity, wavelength_bands),
               bounds=[(x.min().value, x.max().value),
                       (y.min().value, y.max().value)],
               method='Nelder-Mead',
               tol=0.1)
res

Once the iterations completed, the returned object contains the best estimate for the beam center:

In [ ]:
res.x

We can now feed this value again into our `to_q` function, to inspect the $Q$ intensity in all 4 quadrants:

In [ ]:
grouped = to_q(xy=[res.x[0], res.x[1]],
               sample=sample,
               denominator=denominator,
               graph=graph,
               q_bins=q_bins,
               masking_radius=masking_radius,
               gravity=gravity,
               wavelength_bands=wavelength_bands)

pp.plot(grouped, norm='log')

The overlap between the curves is excellent, allowing us to safely perform an azimuthal summation of the counts around the beam center.

As a consistency check, we plot the refined beam center position onto the detector panel image:

In [ ]:
p = sample.sum('tof').copy().hist(y=120, x=128).plot(norm='log', aspect='equal')
p.ax.plot(res.x[0], res.x[1], 'o', color='red', ms=5)
p

## Footnotes

<div id='footnote1'></div>

1. In the full $I(Q)$ reduction, there is a term $D(\lambda)$ in the normalization called the "direct beam" which gives the efficiency of the detectors as a function of wavelength.
Because finding the beam center is required to compute the direct beam in the first place,
we do not include this term in the computation of $I(Q)$ for finding the beam center.
This changes the shape of the $I(Q)$ curve, but since it changes it in the same manner for all $\phi$ angles,
this does not affect the results for finding the beam center.